In [ ]:
import time, pathlib, numpy as np, pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

In [ ]:

DATA_DIR = pathlib.Path("data/processed")
test_df = pd.read_csv(DATA_DIR/"phrasebank_test.csv")

# Map VADER compound to 3 classes
def vader_label(comp):
    if comp >= 0.05:   return "positive"
    if comp <= -0.05:  return "negative"
    return "neutral"

an = SentimentIntensityAnalyzer()
t0 = time.perf_counter()
scores = test_df['sentence'].apply(lambda s: an.polarity_scores(s)['compound'])
preds = scores.map(vader_label)
runtime_s = time.perf_counter() - t0

out = test_df.copy()
out['pred'] = preds
out.to_csv(DATA_DIR/"vader_preds.csv", index=False)

y_true = test_df['label_name']
macro_f1 = f1_score(y_true, preds, average='macro')
acc = accuracy_score(y_true, preds)
print(f"VADER  macro-F1={macro_f1:.3f}  acc={acc:.3f}  runtime={runtime_s:.2f}s")
print(confusion_matrix(y_true, preds, labels=["negative","neutral","positive"]))